
### Импорт библиотек

In [ ]:
import os
import sys
import glob
import random
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import skimage.io                           #Used for imshow function
import skimage.transform                    #Used for resize function
from skimage.morphology import label        #Used for Run-Length-Encoding RLE to create final submission
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import *
import torch
from tensorflow.keras.layers import Input, Conv2D, Lambda, MaxPooling2D, Conv2DTranspose, concatenate, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.activations import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
from tensorflow.keras.layers.experimental.preprocessing import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.optimizers.schedules import *
from tensorflow.keras.backend import clear_session

# SETUP

In [ ]:
TRAIN_PATH = 'train/'
TEST_PATH = 'test/'

IMG_WIDTH       = 256
IMG_HEIGHT      = 256
IMG_SIZE        = 256
IMG_CHANNELS    = 3
input_shape     = (IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS)

BATCH_SIZE      = 32
NUM_EPOCHS      = 100
LR              = 0.001
validation_split= 0.1

SEED = 42
random.seed = SEED
np.random.seed(seed=SEED)

# Загрузка данных

In [ ]:
# распаковываем данные
import zipfile
for name_data in ['test', 'train']:
    tmp_zip = zipfile.ZipFile('../input/'+name_data+'.zip')
    tmp_zip.extractall(name_data)
    tmp_zip.close()

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission.sample(5)

In [ ]:
len(sample_submission)

In [ ]:
train_labels = pd.read_csv('../input/train_labels.csv')
train_labels.sample(5)

In [ ]:
len(train_labels)

In [ ]:
len(train_labels) - train_labels.ImageId.duplicated().sum()

Сверяем по папкам

In [ ]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))
test_ids = next(os.walk(TEST_PATH))

mask_count = 0
for train_id in train_ids[1]:
    masks = next(os.walk(TRAIN_PATH + train_id + '/masks/'))[2]
    mask_count += len(masks)

print('There are {} images.'.format(len(train_ids[1])))
print('There are {} masks.'.format(mask_count))
print('Approximately {} masks per image.'.format(mask_count // len(train_ids[1])))

65 изображений на тесте и 670 на трейне.

Подгрузим и посмотрим картинки.

In [ ]:
def get_X_data(path, output_shape=(None, None)):
    '''
    Loads images from path/{id}/images/{id}.png into a numpy array
    '''
    img_paths = ['{0}/{1}/images/{1}.png'.format(path, id) for id in os.listdir(path)]
    X_data = np.array([skimage.transform.resize(skimage.io.imread(path)[:,:,:3], 
                                                output_shape=output_shape, 
                                                mode='constant', 
                                                preserve_range=True) for path in img_paths], dtype=np.uint8)  #take only 3 channels/bands
    
    return X_data

In [ ]:
%%time
# Get training data
X_train = get_X_data(TRAIN_PATH, output_shape=(IMG_HEIGHT,IMG_WIDTH))
print(X_train.shape, X_train.dtype)

In [ ]:
def get_Y_data(path, output_shape=(None, None)):
    '''
    Loads and concatenates images from path/{id}/masks/{id}.png into a numpy array
    '''
    img_paths = [glob.glob('{0}/{1}/masks/*.png'.format(path, id)) for id in os.listdir(path)]
    
    Y_data = []
    for i, img_masks in enumerate(img_paths):  #loop through each individual nuclei for an image and combine them together
        masks = skimage.io.imread_collection(img_masks).concatenate()  #masks.shape = (num_masks, img_height, img_width)
        mask = np.max(masks, axis=0)                                   #mask.shape = (img_height, img_width)
        mask = skimage.transform.resize(mask, output_shape=output_shape+(1,), mode='constant', preserve_range=True)  #need to add an extra dimension so mask.shape = (img_height, img_width, 1)
        Y_data.append(mask)
    Y_data = np.array(Y_data, dtype=np.bool)
    
    return Y_data

In [ ]:
%%time
# Get training data labels
Y_train = get_Y_data(TRAIN_PATH, output_shape=(IMG_HEIGHT,IMG_WIDTH))
print(Y_train.shape, Y_train.dtype)

In [ ]:
# Check training data
f, axarr = plt.subplots(2,4)
f.set_size_inches(20,10)
ix = random.randint(0, len(train_ids[1]))
axarr[0,0].imshow(X_train[ix])
axarr[0,1].imshow(np.squeeze(Y_train[ix]))

axarr[0,2].imshow(X_train[ix])
axarr[0,3].imshow(np.squeeze(Y_train[ix]))

axarr[1,0].imshow(X_train[ix])
axarr[1,1].imshow(np.squeeze(Y_train[ix]))

axarr[1,2].imshow(X_train[ix])
axarr[1,3].imshow(np.squeeze(Y_train[ix]))

plt.show()

# Подготовка данных

Аугментация

In [ ]:
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

In [ ]:
import albumentations as A

In [ ]:
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=30, interpolation=1, border_mode=4, value=None, mask_value=None, always_apply=False, p=0.5),
#     A.OneOf([
#         A.CenterCrop(height=224, width=200),
#         A.CenterCrop(height=200, width=224),
#     ],p=0.5),
#     A.OneOf([
#         A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
#         A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
#     ],p=0.5),
#     A.GaussianBlur(p=0.05),
#     A.HueSaturationValue(p=0.5),
#     A.RGBShift(),
#     A.FancyPCA(alpha=0.1, always_apply=False, p=0.5),
#     A.Resize(IMG_SIZE, IMG_SIZE)
])

In [ ]:
datagen = ImageDataGenerator(
                                   transform,
                                   #rescale = 1/255,
                                   validation_split = validation_split,
                                   )

test_datagen = ImageDataGenerator(
                                    #rescale = 1/255
                                 )

In [ ]:
datagen.fit(X_train)

train_generator = datagen.flow(
    X_train, 
    Y_train, 
    batch_size=BATCH_SIZE,
    subset='training')

test_generator = datagen.flow(
    X_train, 
    Y_train, 
    batch_size=8,
    subset='validation')

In [ ]:
f, axarr = plt.subplots(2,4)
f.set_size_inches(20,10)

index = random.randint(0, len(train_ids[1]))

axarr[0,0].imshow(X_train[index])
axarr[0,1].imshow(np.squeeze(Y_train[index]))

axarr[0,2].imshow(X_train[index])
axarr[0,3].imshow(np.squeeze(Y_train[index]))

index = random.randint(0, len(train_ids[1]))

axarr[1,0].imshow(X_train[index])
axarr[1,1].imshow(np.squeeze(Y_train[index]))

axarr[1,2].imshow(X_train[index])
axarr[1,3].imshow(np.squeeze(Y_train[index]))

# 2. Build model

Построим U-Net model, по мотивам [U-Net: Convolutional Networks for Biomedical Image Segmentation](https://arxiv.org/pdf/1505.04597.pdf) и близко к этому [репозиторию](https://github.com/jocicmarko/ultrasound-nerve-segmentation) из Kaggle Ultrasound Nerve Segmentation competition.

![](https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/u-net-architecture.png)

Функция ошибки - составной лосс (focal+dice)

In [ ]:
# def combo_loss(y_real, y_pred, eps = 1e-8, gamma = 2):
#   y_pred = torch.sigmoid(y_pred)
#   pt = y_real*y_pred+(1-y_real)*(1-y_pred)
#   focal = (-1*((1-pt)**gamma)*torch.log(pt+eps)).mean()
#   num = 2*torch.sum((y_real*y_pred),dim=(1,2,3))
#   den =  torch.sum((y_real+y_pred),dim=(1,2,3))
#   dice = (1 - num / (den + eps)).mean()
#   return focal + dice

In [ ]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (s)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p1)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p2)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p3)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (p4)
#c5 = BatchNormalization() (c5)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (u9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
optimizer = Adam(ExponentialDecay(LR, 100, 0.9))
model.compile(
    optimizer=optimizer, 
    #loss=combo_loss, 
    loss='binary_crossentropy',
    #loss=tf.keras.metrics.binary_focal_crossentropy,
    #loss = 'sparse_categorical_crossentropy',
    metrics='accuracy'
)
model.summary()

In [ ]:
checkpoint = ModelCheckpoint('best_model.hdf5',
                             monitor='val_accuracy',
                             verbose=1,
                             mode='max',
                             save_best_only=True)
early_stopping = EarlyStopping(monitor='accuracy', 
                               patience=5, 
                               verbose = 1, 
                               restore_best_weights=True)
callbacks_list = [checkpoint, 
                  early_stopping
                 ]

In [ ]:
# Fit model
history = model.fit(
                    train_generator,
                    #test_generator,
                    #X_train, 
                    #Y_train, 
                    steps_per_epoch = len(train_generator), 
                    validation_data = test_generator, 
                    validation_steps = len(test_generator), 
                    #validation_split=validation_split,
                    #batch_size=BATCH_SIZE,
                    epochs = NUM_EPOCHS, 
                    callbacks = callbacks_list,
                    verbose=2
                    )

In [ ]:
model.save('keras_unet.h5')

In [ ]:
def plot_loss_history(history):
    # validation losses
    val_loss = history.history['val_loss']
    loss = history.history['loss']

    plt.title('Loss')
    plt.plot(val_loss, 'r', loss, 'b')
    plt.show()
    
plot_loss_history(history)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs = range(len(acc))
 
plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
 
plt.figure()
 
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
 
plt.show()

# 3. Make predictions

In [ ]:
# Use model to predict train labels
model = load_model('keras_unet.h5',)
Y_predict = model.predict(X_train, verbose=1)
Y_predict.shape

In [ ]:
# Check predict data
f, axarr = plt.subplots(2,3)
f.set_size_inches(20,10)
ix = random.randint(0, len(train_ids[1]))
axarr[0,0].imshow(X_train[ix])
axarr[0,0].set_title('Microscope')
axarr[0,1].imshow(np.squeeze(Y_predict[ix]))
axarr[0,1].set_title('"Predicted" Masks')
axarr[0,2].imshow(np.squeeze(Y_train[ix]))
axarr[0,2].set_title('"GroundTruth" Masks')

axarr[1,0].imshow(X_train[ix])
axarr[1,0].set_title('Microscope')
axarr[1,1].imshow(np.squeeze(Y_predict[ix]))
axarr[1,1].set_title('"Predicted" Masks')
axarr[1,2].imshow(np.squeeze(Y_train[ix]))
axarr[1,2].set_title('"GroundTruth" Masks')

plt.show()

In [ ]:
# Get test data
X_test = get_X_data(TEST_PATH, output_shape=(IMG_HEIGHT,IMG_WIDTH))

# Use model to predict test labels
Y_hat = model.predict(X_test, verbose=1)
Y_hat.shape

In [ ]:
idx = random.randint(0, len(test_ids[1]))
print(X_test[idx].shape)
skimage.io.imshow(X_test[idx])
plt.show()
skimage.io.imshow(Y_hat[idx][:,:,0])
plt.show()

# 4. Encode and Submit

In [ ]:
# Run-length encoding stolen from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

мы ресайзили картинку до 256х256, но чтоб верно предсказать, нам нужно сделать маску под размер изначальной картинки

In [ ]:
# Upsample Y_hat back to the original X_test size (height and width)
Y_hat_upsampled = []
for i, test_id in enumerate(os.listdir(TEST_PATH)):  #loop through test_ids in the test_path
    img = skimage.io.imread('{0}/{1}/images/{1}.png'.format(TEST_PATH, test_id))  #read original test image directly from path
    img_upscaled = skimage.transform.resize(Y_hat[i], (img.shape[0], img.shape[1]), mode='constant', preserve_range=True)  #upscale Y_hat image according to original test image
    Y_hat_upsampled.append(img_upscaled)   #append upscaled image to Y_hat_upsampled
len(Y_hat_upsampled)

Осталось закодировать нашу маску

In [ ]:
# Apply Run-Length Encoding on our Y_hat_upscaled
new_test_ids = []
rles = []
for n, id_ in enumerate(os.listdir(TEST_PATH)):
    rle = list(prob_to_rles(Y_hat_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))
len(new_test_ids)  #note that for each test_image, we can have multiple entries of encoded pixels

In [ ]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()

In [ ]:
len(sub)

In [ ]:
# Clean Folder
# import shutil
# shutil.rmtree('train')
# shutil.rmtree('test')